<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/230_Predictive_Revenue_Gap_Orchestrator_Target_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 📈 From Predicting Outcomes to Influencing Outcomes — The Agent Shift

Traditional data science delivers **insight**, often through dashboards, forecasts, and churn or revenue prediction models. While valuable, these outputs rely on humans to notice patterns, interpret them correctly, and take action — sometimes too late.

Orchestrator AI agents represent the next evolution.

Instead of stopping at *“what might happen?”*, they actively drive business outcomes by:

* continuously monitoring key signals
* detecting risk and opportunity in real time
* prioritizing what matters most
* recommending or automating the next best action
* learning from outcomes to improve future decisions

This shifts the role of data science from **reporting reality** to **shaping reality**.

Where ML models predict revenue leakage, the agent **prevents** it.
Where dashboards highlight churn, the agent **intervenes**.
Where analytics suggest opportunity, the agent **activates** it.

The result is a system that moves organizations from reactive insight to proactive impact — turning intelligence into revenue, retention, and competitive advantage.

This is the strategic leap:
**Data science that doesn’t just explain the business — it runs the business with you.**



# Predictive Revenue Gap Orchestrator Agent

In [ ]:
# ============================================================================
# Predictive Revenue Gap Orchestrator Agent
# ============================================================================

class PredictiveRevenueGapState(TypedDict, total=False):
    """State for Predictive Revenue Gap Orchestrator Agent"""

    # Input
    customer_id: Optional[str]              # Specific customer to analyze (None = all customers)
    analysis_date: Optional[str]            # Date to analyze from (default: latest in data)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                     # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Data Ingestion
    customer_data: Dict[str, Any]           # Loaded customer record (if single customer)
    all_customers: List[Dict[str, Any]]     # All customer records
    sales_history: List[Dict[str, Any]]     # All sales records
    sales_lookup: Dict[str, List[Dict[str, Any]]]  # customer_id -> list of sales records

    # Revenue Analysis
    customer_revenue_baseline: Dict[str, Any]  # Baseline metrics per customer
    # Structure per customer:
    # {
    #   "customer_id": "1",
    #   "total_revenue": 423.15,
    #   "average_weekly_spend": 35.26,
    #   "weeks_active": 12,
    #   "revenue_trend": "declining" | "stable" | "growing",
    #   "recent_weeks_avg": 35.26,  # Last 4 weeks average
    #   "baseline_weeks_avg": 35.26,  # First 4 weeks average
    #   "trend_percentage": -15.2  # Percentage change
    # }

    revenue_predictions: Dict[str, Any]     # Predicted revenue per customer
    # Structure per customer:
    # {
    #   "customer_id": "1",
    #   "predicted_next_week": 30.0,
    #   "predicted_next_month": 120.0,
    #   "prediction_method": "moving_average" | "trend_projection" | "baseline",
    #   "confidence": 0.75  # 0-1 confidence score
    # }

    # Gap Detection
    revenue_gaps: List[Dict[str, Any]]      # Identified revenue gaps
    # Structure per gap:
    # {
    #   "customer_id": "1",
    #   "gap_type": "declining_revenue" | "below_baseline" | "churn_risk" | "zero_spend",
    #   "current_revenue": 30.0,
    #   "expected_revenue": 50.0,
    #   "gap_amount": 20.0,
    #   "gap_percentage": -40.0,
    #   "severity": "high" | "medium" | "low",
    #   "weeks_at_risk": 2,
    #   "rationale": "Customer spend declined 40% from baseline"
    # }

    churn_risk_customers: List[Dict[str, Any]]  # High churn risk customers
    # Structure:
    # {
    #   "customer_id": "1",
    #   "churn_risk_score": 0.85,  # 0-1, higher = more risk
    #   "risk_factors": ["zero_spend_weeks", "declining_trend"],
    #   "weeks_since_last_purchase": 2,
    #   "predicted_churn_probability": 0.75
    # }

    # Opportunity Detection
    revenue_recovery_opportunities: List[Dict[str, Any]]  # Opportunities to close gaps
    # Structure:
    # {
    #   "customer_id": "1",
    #   "opportunity_type": "retention" | "win_back" | "upsell" | "re-engagement",
    #   "potential_revenue": 50.0,
    #   "action_priority": "high" | "medium" | "low",
    #   "recommended_actions": ["loyalty_program", "personalized_offer"],
    #   "rationale": "High-value customer showing decline, immediate retention needed"
    # }

    # Scoring & Ranking
    scored_gaps: List[Dict[str, Any]]       # All gaps with scores
    ranked_gaps: List[Dict[str, Any]]       # Sorted by priority
    top_priority_gaps: List[Dict[str, Any]] # Top N gaps to address

    # Summary Metrics
    gap_summary: Dict[str, Any]
    # Structure:
    # {
    #   "total_customers_analyzed": 200,
    #   "customers_with_gaps": 53,
    #   "total_revenue_gap": 5000.0,
    #   "high_priority_gaps": 15,
    #   "churn_risk_customers": 20,
    #   "potential_recovery_revenue": 7500.0
    # }

    # Output
    revenue_gap_report: str                 # Final markdown report
    report_file_path: Optional[str]          # Path to saved report file

    # Metadata
    errors: List[str]                       # Any errors encountered
    processing_time: Optional[float]       # Time taken to process


@dataclass
class PredictiveRevenueGapConfig:
    """Configuration for Predictive Revenue Gap Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/revenue_gap_reports"

    # Analysis Settings
    baseline_weeks: int = 4                 # Weeks to use for baseline calculation
    recent_weeks: int = 4                    # Recent weeks for trend analysis
    prediction_horizon_weeks: int = 4       # Weeks ahead to predict

    # Gap Detection Thresholds
    gap_thresholds: Dict[str, Any] = field(default_factory=lambda: {
        "declining_revenue_threshold": -15.0,  # % decline to flag
        "below_baseline_threshold": -20.0,     # % below baseline to flag
        "churn_risk_zero_weeks": 2,            # Zero spend weeks to flag churn risk
        "high_severity_gap": -30.0,            # % gap for high severity
        "medium_severity_gap": -15.0           # % gap for medium severity
    })

    # Scoring Weights
    scoring_weights: Dict[str, float] = field(default_factory=lambda: {
        "revenue_impact": 0.35,              # Revenue gap amount
        "churn_risk": 0.30,                 # Churn probability
        "customer_value": 0.20,             # Historical customer value
        "recovery_probability": 0.15         # Likelihood of recovery
    })

    # Top N Selection
    top_n_gaps: int = 20                    # Number of top gaps to prioritize

    # LLM Enhancement (optional, for Phase 8)
    enable_llm_recommendations: bool = False
    llm_recommendation_max_gaps: int = 10    # Max gaps to enhance with LLM


# Data loading utilities for Predictive Revenue Gap Orchestrator

# 📂 Data Loading Utilities — Summary

These utilities handle structured ingestion of customer and sales data for the Predictive Revenue Gap Orchestrator. They provide a clean, consistent interface for downstream analytics, rule-based detection, prediction, and agent orchestration.

The goals of this module are:

✅ keep data loading separate from business logic
✅ ensure utilities are independently testable in Cursor
✅ standardize data shape and typing
✅ support fast lookup for customer-level analysis

---

## ✅ What This Module Does

### 1️⃣ Load Individual Customer Records

Retrieves a single customer’s demographics and loyalty status from `retail_customers.csv`.

Useful for:

* personalized recommendations
* segmentation
* feature enrichment

### 2️⃣ Load All Customers

Reads the full customer population into memory, returning structured dictionaries typed for analysis.

### 3️⃣ Load Full Sales History

Parses weekly transaction-level spend data from `retail_weekly_sales.csv`, converting numeric fields appropriately.

### 4️⃣ Build Customer-Level Sales Lookup

Transforms raw sales records into an optimized dictionary:

```
{ customer_id → [ordered weekly sales records] }
```

This enables fast, repeated access for:

* baseline calculations
* gap detection
* churn scoring
* opportunity generation

---

## ✅ Why It Matters

* predictable input format for every orchestrator node
* removes duplication across utilities
* improves performance during customer-by-customer processing
* supports functional, testable architecture

This keeps the orchestrator clean, modular, and scalable — exactly what you want before moving into LangGraph node wiring.



In [ ]:
"""Data loading utilities for Predictive Revenue Gap Orchestrator

- Utilities are independently testable
- Utilities do the work, nodes orchestrate
"""

import csv
from typing import Dict, List, Any, Optional
from pathlib import Path


def load_customer_data(customer_id: str, data_dir: str = "data") -> Dict[str, Any]:
    """Load customer from CSV file"""
    data_path = Path(data_dir) / "retail_customers.csv"

    with open(data_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['customer_id'] == customer_id:
                # Convert types
                return {
                    "customer_id": row['customer_id'],
                    "age": int(row['age']),
                    "household_size": int(row['household_size']),
                    "loyalty_member": row['loyalty_member'] == 'True'
                }

    raise ValueError(f"Customer {customer_id} not found")


def load_all_customers(data_dir: str = "data") -> List[Dict[str, Any]]:
    """Load all customers from CSV file"""
    data_path = Path(data_dir) / "retail_customers.csv"
    customers = []

    with open(data_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            customers.append({
                "customer_id": row['customer_id'],
                "age": int(row['age']),
                "household_size": int(row['household_size']),
                "loyalty_member": row['loyalty_member'] == 'True'
            })

    return customers


def load_sales_history(data_dir: str = "data") -> List[Dict[str, Any]]:
    """Load all sales history from CSV file"""
    data_path = Path(data_dir) / "retail_weekly_sales.csv"
    sales = []

    with open(data_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            sales.append({
                "customer_id": row['customer_id'],
                "week_start_date": row['week_start_date'],
                "weekly_spend": float(row['weekly_spend'])
            })

    return sales


def build_sales_lookup(sales_history: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """Create fast lookup dictionary: customer_id -> list of sales records"""
    lookup = {}

    for sale in sales_history:
        customer_id = sale['customer_id']
        if customer_id not in lookup:
            lookup[customer_id] = []
        lookup[customer_id].append(sale)

    # Sort each customer's sales by date
    for customer_id in lookup:
        lookup[customer_id].sort(key=lambda x: x['week_start_date'])

    return lookup



# Nodes for Predictive Revenue Gap Orchestrator

---

# 🚨 Revenue Gap Detection Utilities — Summary

This module contains the analytical core of the Predictive Revenue Gap Orchestrator. Its purpose is to proactively **detect early signs of revenue leakage and churn risk** at the individual customer level — before losses appear in financial reporting.

It is intentionally **rule-based and interpretable**, making it ideal for MVP validation, stakeholder trust-building, and iterative refinement.

---

## ✅ What This Module Does

For each customer, it:

1. Analyzes recent spending behavior
2. Compares it against historical baseline expectations
3. Detects meaningful negative changes in revenue
4. Flags potential churn events (e.g., zero-spend weeks)
5. Assigns severity levels and contextual explanations
6. Produces structured outputs for downstream orchestration

This transforms raw weekly sales records into business-ready intelligence.

---

## 🔍 Types of Revenue Gaps Detected

### 1️⃣ Declining Revenue Trend

Triggered when spend is decreasing consistently over time and exceeds a configurable threshold (default: −15%).

Signals:

* competitor defection
* tightening household budgets
* dissatisfaction or reduced shopping frequency

Outputs include:

* % decline
* revenue gap amount
* severity (low/medium/high)
* weeks exhibiting decline

---

### 2️⃣ Below-Baseline Weekly Spend

Activated when the most recent week is significantly lower than historical average (default: −20%).

Useful for:

* short-term behavioral dips
* early warning before full churn
* seasonal pattern deviations

---

### 3️⃣ Zero-Spend / Silent Churn Risk

Identifies customers who have stopped purchasing altogether.

Triggered when the customer has multiple zero-spend weeks within the recent window (default: ≥2 of last 4 weeks).

This is automatically classified as **high severity**.

---

## 📉 Churn Risk Scoring

Beyond gap detection, the module also computes a lightweight churn risk score based on:

* number of recent zero-spend weeks
* declining trend severity
* ratio of recent spend to baseline average
* weeks since last purchase

It outputs:

* churn risk score (0.0–1.0)
* predicted churn probability
* human-readable risk factors
* recency indicator

This helps prioritize outreach and marketing spend.

---

## 🎯 Output Structure

The top-level function returns:

```python
{
  "revenue_gaps": [...],          # structured gap insights
  "churn_risk_customers": [...]   # prioritized retention list
}
```

These results directly feed:

* opportunity detection
* recommendation engines
* LangGraph action nodes
* dashboards or alerts
* CRM automation workflows

---

## ✅ Why This Utility Matters

This module enables businesses to:

✅ detect revenue declines early
✅ quantify financial exposure
✅ avoid reactive decision-making
✅ focus retention efforts where it matters
✅ operationalize customer-level intelligence

It moves organizations from *“We lost revenue”* → *“We prevented revenue loss.”*

---

## 🧩 Engineering Benefits

* independently testable utility
* decoupled from orchestration logic
* configurable thresholds
* transparent business rules
* production-friendly and explainable
* easy to enhance or extend with ML later

Perfect foundation for a scalable agent architecture.




In [ ]:
"""Nodes for Predictive Revenue Gap Orchestrator

- Nodes are thin - they orchestrate, utilities do the work
- Each node has one responsibility
- Linear workflow: Goal → Planning → Data Loading → Analysis → Detection → Scoring → Ranking → Reporting
"""

from typing import Dict, Any
from config import PredictiveRevenueGapState
from .utilities.data_loading import (
    load_customer_data,
    load_all_customers,
    load_sales_history,
    build_sales_lookup
)


def goal_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for revenue gap analysis.

    This is a simple rule-based goal definition that sets the framework.
    """
    customer_id = state.get("customer_id")

    if not customer_id:
        return {
            "errors": state.get("errors", []) + ["goal_node: customer_id is required"]
        }

    goal = {
        "objective": "Identify and prioritize revenue gaps and churn risks",
        "customer_id": customer_id,
        "focus_areas": [
            "revenue_baseline_analysis",
            "revenue_prediction",
            "gap_detection",
            "churn_risk_assessment",
            "recovery_opportunities"
        ]
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load customer data and sales history",
            "dependencies": [],
            "outputs": ["customer_data", "sales_history", "sales_lookup"]
        },
        {
            "step": 2,
            "name": "revenue_analysis",
            "description": "Calculate baseline revenue and trends",
            "dependencies": ["data_loading"],
            "outputs": ["customer_revenue_baseline", "revenue_predictions"]
        },
        {
            "step": 3,
            "name": "gap_detection",
            "description": "Identify revenue gaps and churn risks",
            "dependencies": ["revenue_analysis"],
            "outputs": ["revenue_gaps", "churn_risk_customers"]
        },
        {
            "step": 4,
            "name": "opportunity_detection",
            "description": "Identify recovery opportunities",
            "dependencies": ["gap_detection"],
            "outputs": ["revenue_recovery_opportunities"]
        },
        {
            "step": 5,
            "name": "scoring",
            "description": "Score gaps by priority",
            "dependencies": ["opportunity_detection"],
            "outputs": ["scored_gaps"]
        },
        {
            "step": 6,
            "name": "ranking",
            "description": "Rank gaps and select top priorities",
            "dependencies": ["scoring"],
            "outputs": ["ranked_gaps", "top_priority_gaps", "gap_summary"]
        },
        {
            "step": 7,
            "name": "report_generation",
            "description": "Generate final revenue gap report",
            "dependencies": ["ranking"],
            "outputs": ["revenue_gap_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading customer and sales data.
    """
    errors = state.get("errors", [])
    customer_id = state.get("customer_id")
    data_dir = state.get("data_dir", "data")

    if not customer_id:
        return {
            "errors": errors + ["data_loading_node: customer_id is required"]
        }

    try:
        # Load customer data
        customer_data = load_customer_data(customer_id, data_dir)

        # Load all customers (for comparison/analysis)
        all_customers = load_all_customers(data_dir)

        # Load sales history
        sales_history = load_sales_history(data_dir)

        # Build lookup for fast access
        sales_lookup = build_sales_lookup(sales_history)

        return {
            "customer_data": customer_data,
            "all_customers": all_customers,
            "sales_history": sales_history,
            "sales_lookup": sales_lookup,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }



# Revenue analysis utilities for Predictive Revenue Gap Orchestrator

In [ ]:
"""Revenue analysis utilities for Predictive Revenue Gap Orchestrator

- Utilities are independently testable
- Rule-based calculations (no LLM needed for MVP)
"""

from typing import Dict, List, Any
from datetime import datetime, timedelta


def calculate_revenue_baseline(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    baseline_weeks: int = 4
) -> Dict[str, Any]:
    """
    Calculate baseline revenue metrics for a customer.

    Args:
        customer_id: Customer identifier
        sales_records: List of sales records sorted by date
        baseline_weeks: Number of weeks to use for baseline calculation

    Returns:
        Dictionary with baseline metrics
    """
    if not sales_records:
        return {
            "customer_id": customer_id,
            "total_revenue": 0.0,
            "average_weekly_spend": 0.0,
            "weeks_active": 0,
            "revenue_trend": "no_data",
            "recent_weeks_avg": 0.0,
            "baseline_weeks_avg": 0.0,
            "trend_percentage": 0.0
        }

    # Calculate total revenue
    total_revenue = sum(record['weekly_spend'] for record in sales_records)
    weeks_active = len(sales_records)
    average_weekly_spend = total_revenue / weeks_active if weeks_active > 0 else 0.0

    # Calculate baseline (first N weeks)
    baseline_records = sales_records[:baseline_weeks] if len(sales_records) >= baseline_weeks else sales_records
    baseline_total = sum(record['weekly_spend'] for record in baseline_records)
    baseline_weeks_avg = baseline_total / len(baseline_records) if baseline_records else 0.0

    # Calculate recent average (last N weeks)
    recent_records = sales_records[-baseline_weeks:] if len(sales_records) >= baseline_weeks else sales_records
    recent_total = sum(record['weekly_spend'] for record in recent_records)
    recent_weeks_avg = recent_total / len(recent_records) if recent_records else 0.0

    # Calculate trend
    if baseline_weeks_avg > 0:
        trend_percentage = ((recent_weeks_avg - baseline_weeks_avg) / baseline_weeks_avg) * 100
    else:
        trend_percentage = 0.0

    # Determine trend direction
    if trend_percentage > 5.0:
        revenue_trend = "growing"
    elif trend_percentage < -5.0:
        revenue_trend = "declining"
    else:
        revenue_trend = "stable"

    return {
        "customer_id": customer_id,
        "total_revenue": round(total_revenue, 2),
        "average_weekly_spend": round(average_weekly_spend, 2),
        "weeks_active": weeks_active,
        "revenue_trend": revenue_trend,
        "recent_weeks_avg": round(recent_weeks_avg, 2),
        "baseline_weeks_avg": round(baseline_weeks_avg, 2),
        "trend_percentage": round(trend_percentage, 2)
    }


def predict_revenue(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    baseline: Dict[str, Any],
    prediction_horizon_weeks: int = 4
) -> Dict[str, Any]:
    """
    Predict future revenue for a customer.

    MVP: Simple rule-based prediction using moving average and trend.

    Args:
        customer_id: Customer identifier
        sales_records: List of sales records sorted by date
        baseline: Baseline metrics from calculate_revenue_baseline
        prediction_horizon_weeks: Number of weeks ahead to predict

    Returns:
        Dictionary with predictions
    """
    if not sales_records:
        return {
            "customer_id": customer_id,
            "predicted_next_week": 0.0,
            "predicted_next_month": 0.0,
            "prediction_method": "no_data",
            "confidence": 0.0
        }

    # Use recent average as base prediction
    recent_avg = baseline.get("recent_weeks_avg", 0.0)
    trend_percentage = baseline.get("trend_percentage", 0.0)

    # Simple trend projection: apply trend to recent average
    # Cap trend impact at ±20% to avoid extreme predictions
    trend_factor = max(-0.2, min(0.2, trend_percentage / 100))
    predicted_next_week = recent_avg * (1 + trend_factor)

    # Predict next month (4 weeks)
    predicted_next_month = predicted_next_week * prediction_horizon_weeks

    # Confidence based on data quality and consistency
    weeks_active = baseline.get("weeks_active", 0)
    if weeks_active >= 8:
        confidence = 0.8
    elif weeks_active >= 4:
        confidence = 0.6
    else:
        confidence = 0.4

    # Lower confidence if high volatility
    if weeks_active > 0:
        spends = [r['weekly_spend'] for r in sales_records]
        if len(spends) > 1:
            avg = sum(spends) / len(spends)
            variance = sum((s - avg) ** 2 for s in spends) / len(spends)
            std_dev = variance ** 0.5
            if std_dev > avg * 0.5:  # High volatility
                confidence *= 0.8

    prediction_method = "trend_projection" if abs(trend_percentage) > 5 else "moving_average"

    return {
        "customer_id": customer_id,
        "predicted_next_week": round(predicted_next_week, 2),
        "predicted_next_month": round(predicted_next_month, 2),
        "prediction_method": prediction_method,
        "confidence": round(confidence, 2)
    }


def analyze_all_customers_revenue(
    all_customers: List[Dict[str, Any]],
    sales_lookup: Dict[str, List[Dict[str, Any]]],
    baseline_weeks: int = 4,
    prediction_horizon_weeks: int = 4
) -> Dict[str, Any]:
    """
    Analyze revenue for all customers.

    Returns:
        Dictionary with:
        - customer_revenue_baseline: Dict[customer_id, baseline_metrics]
        - revenue_predictions: Dict[customer_id, prediction_metrics]
    """
    customer_revenue_baseline = {}
    revenue_predictions = {}

    for customer in all_customers:
        customer_id = customer['customer_id']
        sales_records = sales_lookup.get(customer_id, [])

        # Calculate baseline
        baseline = calculate_revenue_baseline(customer_id, sales_records, baseline_weeks)
        customer_revenue_baseline[customer_id] = baseline

        # Predict revenue
        prediction = predict_revenue(customer_id, sales_records, baseline, prediction_horizon_weeks)
        revenue_predictions[customer_id] = prediction

    return {
        "customer_revenue_baseline": customer_revenue_baseline,
        "revenue_predictions": revenue_predictions
    }



# Gap detection utilities for Predictive Revenue Gap Orchestrator
---

# 🧠 Predictive Revenue Gap Detection — Business Logic Overview

This module implements the core intelligence of the **Predictive Revenue Gap Orchestrator** — a system designed to proactively identify revenue leakage, emerging churn risk, and customers requiring intervention.

It doesn’t rely on machine learning for the MVP.
Instead, it applies **interpretable, rule-based analytics**, making results explainable, auditable, and business-friendly.

---

## ✅ What This Utility Does

For each customer, the orchestrator:

1. **Analyzes recent spending behavior**
2. **Compares performance against historical baseline**
3. **Evaluates predicted revenue vs. actual trends**
4. **Applies business thresholds to detect risks**
5. **Quantifies financial gaps and severity**
6. **Assesses churn likelihood**
7. **Returns structured insights for downstream orchestration**

This transforms raw weekly sales data into **actionable revenue intelligence**.

---

## 🔍 Revenue Gap Detection Logic

The system currently detects three high-value leakage patterns:

### 1️⃣ Declining Revenue Trend

Triggered when:

* customer’s average recent spend is significantly lower than historical baseline
* negative trend exceeds threshold (default: −15%)

Outputs include:

* % decline
* estimated revenue gap
* severity label (low / medium / high)
* weeks showing deterioration
* human-readable rationale

**Business meaning:** customer is gradually reducing spend — potential competitor migration or budget tightening.

---

### 2️⃣ Below Baseline Performance

Flags situations where:

* latest weekly spend is sharply below normal
* short-term drop exceeds threshold (default: −20%)

Useful for sudden behavior shifts, operational issues, or life-event changes.

---

### 3️⃣ Zero-Spend / Churn Risk

Identifies **silent churn** by checking:

* number of consecutive zero-spend weeks
* threshold default: 2 out of last 4

Automatically treated as **high severity**.

---

## 📉 Churn Risk Assessment Logic

Instead of a single label, the system scores churn probability using multiple behavioral signals:

* repeated zero-spend weeks
* steep declining trend
* recent average spend < 50% of baseline

Each contributes to a weighted **churn risk score (0.0–1.0)**, which is then mapped to a predicted churn probability.

Also returns:

* risk factors driving the score
* weeks since last purchase

**Why it matters:** supports personalized, targeted retention actions.

---

## 🎯 Output Structure

The orchestration layer receives two actionable collections:

* `revenue_gaps`: customers with detected leakage patterns
* `churn_risk_customers`: prioritized retention list

These can be routed into:

* alerting systems
* CRM tasks
* marketing automation
* business dashboards
* LLM action-recommendation nodes

---

## 💼 Business Value

This utility enables companies to:

✅ detect revenue loss **before** it appears in financials
✅ focus retention efforts on the highest-risk customers
✅ quantify the financial impact of inaction
✅ transition from reactive reporting → proactive orchestration
✅ maintain interpretability for stakeholders (finance, execs, ops)

It turns weekly sales data into **early-warning signals and decision leverage**, not just historical reporting.

---

## 🧱 Why This Is Great for an MVP

* simple thresholds = fast iteration
* no ML dependency
* easily tuned per business
* transparent + explainable
* modular — tested independently before orchestration
* extendable into LLM + ML + automation later




In [ ]:
"""Gap detection utilities for Predictive Revenue Gap Orchestrator

- Utilities are independently testable
- Rule-based gap detection (no LLM needed for MVP)
"""

from typing import Dict, List, Any


def detect_revenue_gaps(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    baseline: Dict[str, Any],
    prediction: Dict[str, Any],
    gap_thresholds: Dict[str, Any]
) -> List[Dict[str, Any]]:
    """
    Detect revenue gaps for a customer.

    Args:
        customer_id: Customer identifier
        sales_records: List of sales records sorted by date
        baseline: Baseline metrics
        prediction: Revenue predictions
        gap_thresholds: Thresholds for gap detection

    Returns:
        List of detected gaps
    """
    gaps = []

    if not sales_records:
        return gaps

    # Get recent spend (last week)
    recent_spend = sales_records[-1]['weekly_spend'] if sales_records else 0.0
    baseline_avg = baseline.get("baseline_weeks_avg", 0.0)
    recent_avg = baseline.get("recent_weeks_avg", 0.0)
    trend_percentage = baseline.get("trend_percentage", 0.0)
    predicted_next_week = prediction.get("predicted_next_week", 0.0)

    # Gap 1: Declining Revenue
    declining_threshold = gap_thresholds.get("declining_revenue_threshold", -15.0)
    if trend_percentage < declining_threshold:
        gap_amount = recent_avg - baseline_avg
        gap_percentage = trend_percentage

        # Determine severity
        if gap_percentage < gap_thresholds.get("high_severity_gap", -30.0):
            severity = "high"
        elif gap_percentage < gap_thresholds.get("medium_severity_gap", -15.0):
            severity = "medium"
        else:
            severity = "low"

        gaps.append({
            "customer_id": customer_id,
            "gap_type": "declining_revenue",
            "current_revenue": round(recent_avg, 2),
            "expected_revenue": round(baseline_avg, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(gap_percentage, 2),
            "severity": severity,
            "weeks_at_risk": len([r for r in sales_records[-4:] if r['weekly_spend'] < baseline_avg * 0.85]),
            "rationale": f"Customer spend declined {abs(gap_percentage):.1f}% from baseline"
        })

    # Gap 2: Below Baseline
    below_baseline_threshold = gap_thresholds.get("below_baseline_threshold", -20.0)
    if recent_spend > 0 and baseline_avg > 0:
        below_percentage = ((recent_spend - baseline_avg) / baseline_avg) * 100
        if below_percentage < below_baseline_threshold:
            gap_amount = recent_spend - baseline_avg

            if below_percentage < gap_thresholds.get("high_severity_gap", -30.0):
                severity = "high"
            elif below_percentage < gap_thresholds.get("medium_severity_gap", -15.0):
                severity = "medium"
            else:
                severity = "low"

            gaps.append({
                "customer_id": customer_id,
                "gap_type": "below_baseline",
                "current_revenue": round(recent_spend, 2),
                "expected_revenue": round(baseline_avg, 2),
                "gap_amount": round(gap_amount, 2),
                "gap_percentage": round(below_percentage, 2),
                "severity": severity,
                "weeks_at_risk": 1,
                "rationale": f"Recent spend ${recent_spend:.2f} is {abs(below_percentage):.1f}% below baseline ${baseline_avg:.2f}"
            })

    # Gap 3: Zero Spend (Churn Risk)
    zero_weeks_threshold = gap_thresholds.get("churn_risk_zero_weeks", 2)
    recent_zero_weeks = sum(1 for r in sales_records[-4:] if r['weekly_spend'] == 0.0)

    if recent_zero_weeks >= zero_weeks_threshold:
        gap_amount = 0.0 - baseline_avg

        gaps.append({
            "customer_id": customer_id,
            "gap_type": "zero_spend",
            "current_revenue": 0.0,
            "expected_revenue": round(baseline_avg, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": -100.0 if baseline_avg > 0 else 0.0,
            "severity": "high",
            "weeks_at_risk": recent_zero_weeks,
            "rationale": f"Customer has {recent_zero_weeks} zero-spend weeks in recent period (churn risk)"
        })

    return gaps


def assess_churn_risk(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    baseline: Dict[str, Any],
    gap_thresholds: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Assess churn risk for a customer.

    Returns:
        Dictionary with churn risk assessment
    """
    if not sales_records:
        return {
            "customer_id": customer_id,
            "churn_risk_score": 0.0,
            "risk_factors": [],
            "weeks_since_last_purchase": 0,
            "predicted_churn_probability": 0.0
        }

    risk_factors = []
    risk_score = 0.0

    # Factor 1: Zero spend weeks
    recent_zero_weeks = sum(1 for r in sales_records[-4:] if r['weekly_spend'] == 0.0)
    zero_weeks_threshold = gap_thresholds.get("churn_risk_zero_weeks", 2)

    if recent_zero_weeks >= zero_weeks_threshold:
        risk_factors.append("zero_spend_weeks")
        risk_score += 0.4

    # Factor 2: Declining trend
    trend_percentage = baseline.get("trend_percentage", 0.0)
    if trend_percentage < -20.0:
        risk_factors.append("declining_trend")
        risk_score += 0.3
    elif trend_percentage < -10.0:
        risk_factors.append("declining_trend")
        risk_score += 0.15

    # Factor 3: Recent spend vs baseline
    recent_avg = baseline.get("recent_weeks_avg", 0.0)
    baseline_avg = baseline.get("baseline_weeks_avg", 0.0)

    if baseline_avg > 0:
        decline_ratio = recent_avg / baseline_avg
        if decline_ratio < 0.5:
            risk_factors.append("significant_decline")
            risk_score += 0.3

    # Cap risk score at 1.0
    risk_score = min(1.0, risk_score)

    # Calculate weeks since last purchase (non-zero)
    weeks_since_last_purchase = 0
    for record in reversed(sales_records):
        if record['weekly_spend'] > 0:
            break
        weeks_since_last_purchase += 1

    # Predicted churn probability (simple rule-based)
    if risk_score >= 0.7:
        predicted_churn_probability = 0.75
    elif risk_score >= 0.5:
        predicted_churn_probability = 0.50
    elif risk_score >= 0.3:
        predicted_churn_probability = 0.25
    else:
        predicted_churn_probability = 0.10

    return {
        "customer_id": customer_id,
        "churn_risk_score": round(risk_score, 2),
        "risk_factors": risk_factors,
        "weeks_since_last_purchase": weeks_since_last_purchase,
        "predicted_churn_probability": round(predicted_churn_probability, 2)
    }


def detect_all_gaps(
    all_customers: List[Dict[str, Any]],
    sales_lookup: Dict[str, List[Dict[str, Any]]],
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    revenue_predictions: Dict[str, Dict[str, Any]],
    gap_thresholds: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Detect gaps and churn risks for all customers.

    Returns:
        Dictionary with:
        - revenue_gaps: List of all detected gaps
        - churn_risk_customers: List of customers with churn risk
    """
    revenue_gaps = []
    churn_risk_customers = []

    for customer in all_customers:
        customer_id = customer['customer_id']
        sales_records = sales_lookup.get(customer_id, [])
        baseline = customer_revenue_baseline.get(customer_id, {})
        prediction = revenue_predictions.get(customer_id, {})

        # Detect gaps
        gaps = detect_revenue_gaps(customer_id, sales_records, baseline, prediction, gap_thresholds)
        revenue_gaps.extend(gaps)

        # Assess churn risk
        churn_assessment = assess_churn_risk(customer_id, sales_records, baseline, gap_thresholds)
        if churn_assessment["churn_risk_score"] >= 0.3:
            churn_risk_customers.append(churn_assessment)

    return {
        "revenue_gaps": revenue_gaps,
        "churn_risk_customers": churn_risk_customers
    }



# Opportunity detection utilities for Predictive Revenue Gap Orchestrator

---

# 🌱 Opportunity Detection Utilities — Business Logic Overview

While gap detection identifies **where revenue is leaking**, the opportunity detection layer determines **how much revenue could be recovered and what action should be taken**. It transforms risk signals into business opportunities.

Instead of simply flagging declining customers, this utility answers:

✅ *Is this worth acting on?*
✅ *What type of opportunity is it?*
✅ *How much revenue could be saved or regained?*
✅ *What actions should the agent recommend?*
✅ *How urgent is intervention?*

This is what turns an insights engine into a **revenue orchestration agent**.

---

## ✅ What This Utility Does

For every detected revenue gap, the system:

1. Classifies the type of opportunity
2. Quantifies potential recoverable revenue
3. Prioritizes action urgency
4. Suggests targeted interventions
5. Incorporates customer characteristics (e.g., loyalty membership)
6. Leverages churn risk assessments to escalate priority

The output becomes the fuel for personalized, automated revenue recovery workflows.

---

## 🔍 Opportunity Classification Logic

Based on the type and severity of revenue gap, opportunities fall into four categories:

### 1️⃣ Win-Back Opportunity

Triggered by:

* multiple zero-spend weeks → likely churn

Recommended actions:

* win-back campaign
* targeted incentive
* loyalty program re-activation

Business intent:
➡️ recover customers already leaving

---

### 2️⃣ Retention Opportunity

Triggered by:

* high-severity declining spend
* multi-week downward trend

Recommended actions:

* personalized offers
* value reinforcement messaging
* loyalty engagement

Business intent:
➡️ prevent future churn and protect predictable revenue

---

### 3️⃣ Re-Engagement Opportunity

Triggered by:

* recent spend significantly below baseline
* short-term behavioral dip

Recommended actions:

* gentle outreach
* reminder campaigns
* category-based promo

Business intent:
➡️ nudge returning behavior before it becomes churn

---

### 4️⃣ General At-Risk Opportunity

Fallback for lower-severity gaps where proactive monitoring is still valuable.

---

## 💰 Potential Revenue Estimation

The system estimates financial upside by multiplying the weekly revenue gap over a realistic recovery window:

* Zero-spend: baseline × 4 weeks
* Declining/Below-baseline: gap amount × 4 weeks
* Low-severity gaps: gap amount × 2 weeks

This helps prioritize highest-ROI interventions.

---

## 🎯 Action Prioritization

Opportunity urgency is determined by:

* gap severity (high / medium / low)
* churn-risk score (≥ 0.5 escalates priority)
* customer value signals (e.g., loyalty member)

Priority levels:

* **high** — act immediately
* **medium** — monitor + intervene soon
* **low** — awareness only

This prevents teams from overreacting or underreacting.

---

## 🧩 Personalized Action Recommendations

Suggested interventions adapt based on:

* gap type
* severity
* customer loyalty status

Examples:

* send loyalty perks if already a member
* recommend joining loyalty program if not
* choose win-back vs. re-engagement vs. retention campaigns

This ensures recommended actions feel relevant, not generic.

---

## 🧠 Why This Utility Is Valuable

Businesses don’t just want alerts — they want **next steps**.

This module provides:

✅ a quantified business case
✅ a clear call to action
✅ prioritization for limited resources
✅ customer-specific retention strategy
✅ structured data ready for automation

It bridges analytics → operations → revenue outcomes.

---

## 🔁 How It Fits Into the Orchestrator

Pipeline flow:

1. sales data → gap detection
2. gap detection → churn scoring
3. churn scoring + gap → opportunity detection ✅
4. opportunity → LLM/automation → recommended action
5. orchestrator executes or routes to teams



In [ ]:
"""Opportunity detection utilities for Predictive Revenue Gap Orchestrator

Following ORCHESTRATOR_AGENTS_GUIDE_3.md pattern:
- Utilities are independently testable
- Rule-based opportunity detection (no LLM needed for MVP)
"""

from typing import Dict, List, Any, Optional


def identify_recovery_opportunities(
    gap: Dict[str, Any],
    customer_data: Dict[str, Any],
    baseline: Dict[str, Any],
    churn_risk: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Identify recovery opportunities for a revenue gap.

    Args:
        gap: Revenue gap dictionary
        customer_data: Customer demographic data
        baseline: Revenue baseline metrics
        churn_risk: Churn risk assessment (optional)

    Returns:
        Opportunity dictionary
    """
    customer_id = gap["customer_id"]
    gap_type = gap["gap_type"]
    gap_amount = abs(gap["gap_amount"])
    severity = gap["severity"]

    # Determine opportunity type based on gap type
    if gap_type == "zero_spend":
        opportunity_type = "win_back"
        potential_revenue = baseline.get("baseline_weeks_avg", 0.0) * 4  # 1 month recovery
        recommended_actions = ["re-engagement_campaign", "win_back_offer", "loyalty_program"]
    elif gap_type == "declining_revenue" and severity == "high":
        opportunity_type = "retention"
        potential_revenue = gap_amount * 4  # 1 month recovery
        recommended_actions = ["retention_campaign", "personalized_offer", "loyalty_program"]
    elif gap_type == "below_baseline":
        opportunity_type = "re-engagement"
        potential_revenue = gap_amount * 4
        recommended_actions = ["re-engagement_campaign", "personalized_offer"]
    else:
        opportunity_type = "retention"
        potential_revenue = gap_amount * 2  # Conservative estimate
        recommended_actions = ["retention_campaign"]

    # Adjust actions based on customer characteristics
    if customer_data.get("loyalty_member"):
        recommended_actions.append("loyalty_benefits")

    # Determine action priority
    if severity == "high" or (churn_risk and churn_risk.get("churn_risk_score", 0) >= 0.7):
        action_priority = "high"
    elif severity == "medium" or (churn_risk and churn_risk.get("churn_risk_score", 0) >= 0.5):
        action_priority = "medium"
    else:
        action_priority = "low"

    # Create rationale
    rationale = f"{severity.capitalize()}-priority {opportunity_type.replace('_', ' ')} opportunity. "
    rationale += gap.get("rationale", "")
    if churn_risk and churn_risk.get("churn_risk_score", 0) >= 0.5:
        rationale += f" High churn risk ({churn_risk['churn_risk_score']:.0%})."

    return {
        "customer_id": customer_id,
        "opportunity_type": opportunity_type,
        "potential_revenue": round(potential_revenue, 2),
        "action_priority": action_priority,
        "recommended_actions": recommended_actions,
        "rationale": rationale
    }


def detect_all_opportunities(
    revenue_gaps: List[Dict[str, Any]],
    all_customers: List[Dict[str, Any]],
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    churn_risk_customers: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Detect recovery opportunities for all gaps.

    Returns:
        List of recovery opportunities
    """
    # Create lookup dictionaries
    customer_lookup = {c["customer_id"]: c for c in all_customers}
    baseline_lookup = customer_revenue_baseline
    churn_risk_lookup = {c["customer_id"]: c for c in churn_risk_customers}

    opportunities = []

    for gap in revenue_gaps:
        customer_id = gap["customer_id"]
        customer_data = customer_lookup.get(customer_id, {})
        baseline = baseline_lookup.get(customer_id, {})
        churn_risk = churn_risk_lookup.get(customer_id)

        opportunity = identify_recovery_opportunities(
            gap,
            customer_data,
            baseline,
            churn_risk
        )
        opportunities.append(opportunity)

    return opportunities



# Scoring utilities for Predictive Revenue Gap Orchestrator



---

# 🎯 Scoring Utilities — Summary

Once revenue gaps and recovery opportunities have been identified, the orchestrator must determine **which customers the business should act on first**. The scoring utilities provide that prioritization layer.

This module converts qualitative signals — severity, churn risk, customer value, and potential recovery — into a **single, comparable numerical score**, enabling ranking, triage, and automated decision-making.

---

## ✅ What This Module Does

For every detected revenue gap, it:

1. Evaluates financial impact
2. Incorporates churn risk likelihood
3. Considers customer lifetime value
4. Assesses probability of successful recovery
5. Applies configurable weighting
6. Produces a final prioritization score (0–10)

This transforms raw gap information into **business-actionable scoring intelligence**.

---

## 🧠 Scoring Factors Explained

Each gap is scored across four dimensions — each 0–10:

### 1️⃣ Revenue Impact Score

Measures how large the gap is relative to the customer’s historical revenue.

➡️ Prioritizes high-value losses over small fluctuations.

### 2️⃣ Churn Risk Score

Derived from prior churn risk assessment.

➡️ Customers likely to leave receive higher priority.

### 3️⃣ Customer Value Score

Based on cumulative revenue over time.

➡️ Protects profitable, loyal, or strategic customers.

### 4️⃣ Recovery Probability Score

Estimated ease of winning back the revenue based on opportunity type and severity.

➡️ Helps allocate resources efficiently.

---

## ⚖️ Weighted Final Score

The final prioritization score uses configurable weights:

```python
revenue_impact  = 0.35
churn_risk      = 0.30
customer_value  = 0.20
recovery_prob   = 0.15
```

These weights can be tuned to match business strategy, seasonality, or resource constraints.

The output `final_score` becomes the orchestrator’s basis for:

* ranking customers
* selecting intervention cohorts
* triggering campaigns
* routing to sales or support
* determining urgency

---

## 📦 Output Structure

Each scored gap includes:

```python
{
  "customer_id": ...,
  "gap_type": ...,
  "final_score": 7.85,
  "revenue_impact_score": ...,
  "churn_risk_score": ...,
  "customer_value_score": ...,
  "recovery_probability_score": ...,
  "opportunity": {...}
}
```

This design keeps the scoring transparent, traceable, and debuggable.

---

## 💼 Business Value

This utility enables organizations to:

✅ focus attention — not just awareness
✅ allocate retention budgets strategically
✅ prevent overreacting to low-impact gaps
✅ act faster and with confidence
✅ measure ROI of interventions

It turns a long list of revenue risks into a **prioritized action roadmap**.

---

## 🧱 Engineering Advantages

* no ML dependency — great for MVP
* deterministic, explainable, auditable
* independently testable
* configurable scoring weights
* easily extendable to ML or LLM reasoning later
* plugs directly into LangGraph orchestration nodes

---

## 🚀 Why It Matters in the Agent Workflow

Without scoring:

* the agent knows where gaps exist
  With scoring:
* the agent knows **which gaps matter most right now**

This is what enables automation, alerting, batching, and strategic intervention decisions.



In [ ]:
"""Scoring utilities for Predictive Revenue Gap Orchestrator

Following ORCHESTRATOR_AGENTS_GUIDE_3.md pattern:
- Utilities are independently testable
- Rule-based scoring (no LLM needed for MVP)
"""

from typing import Dict, List, Any, Optional


def score_gap(
    gap: Dict[str, Any],
    opportunity: Dict[str, Any],
    baseline: Dict[str, Any],
    churn_risk: Optional[Dict[str, Any]],
    scoring_weights: Dict[str, float]
) -> Dict[str, Any]:
    """
    Score a revenue gap based on multiple factors.

    Args:
        gap: Revenue gap dictionary
        opportunity: Recovery opportunity dictionary
        baseline: Revenue baseline metrics
        churn_risk: Churn risk assessment (optional)
        scoring_weights: Weights for different scoring factors

    Returns:
        Gap dictionary with scores added
    """
    # Extract values
    gap_amount = abs(gap.get("gap_amount", 0.0))
    gap_percentage = abs(gap.get("gap_percentage", 0.0))
    severity = gap.get("severity", "low")
    potential_revenue = opportunity.get("potential_revenue", 0.0)
    total_revenue = baseline.get("total_revenue", 0.0)

    # Factor 1: Revenue Impact Score (0-10)
    # Based on gap amount relative to customer's total revenue
    if total_revenue > 0:
        impact_ratio = gap_amount / total_revenue
        revenue_impact_score = min(10.0, impact_ratio * 100)
    else:
        revenue_impact_score = min(10.0, gap_amount / 10.0)  # Normalize by $10

    # Factor 2: Churn Risk Score (0-10)
    if churn_risk:
        churn_risk_score = churn_risk.get("churn_risk_score", 0.0) * 10.0
    else:
        churn_risk_score = 0.0

    # Factor 3: Customer Value Score (0-10)
    # Based on historical total revenue
    if total_revenue >= 1000:
        customer_value_score = 10.0
    elif total_revenue >= 500:
        customer_value_score = 7.0
    elif total_revenue >= 200:
        customer_value_score = 5.0
    else:
        customer_value_score = 3.0

    # Factor 4: Recovery Probability Score (0-10)
    # Based on opportunity type and severity
    opportunity_type = opportunity.get("opportunity_type", "")
    if opportunity_type == "win_back":
        recovery_probability_score = 6.0  # Harder to win back
    elif opportunity_type == "retention" and severity == "high":
        recovery_probability_score = 8.0  # High priority retention
    elif opportunity_type == "retention":
        recovery_probability_score = 7.0
    else:
        recovery_probability_score = 6.0

    # Apply weights and calculate final score
    final_score = (
        revenue_impact_score * scoring_weights.get("revenue_impact", 0.35) +
        churn_risk_score * scoring_weights.get("churn_risk", 0.30) +
        customer_value_score * scoring_weights.get("customer_value", 0.20) +
        recovery_probability_score * scoring_weights.get("recovery_probability", 0.15)
    )

    # Add scores to gap dictionary
    scored_gap = gap.copy()
    scored_gap.update({
        "revenue_impact_score": round(revenue_impact_score, 2),
        "churn_risk_score": round(churn_risk_score, 2),
        "customer_value_score": round(customer_value_score, 2),
        "recovery_probability_score": round(recovery_probability_score, 2),
        "final_score": round(final_score, 2),
        "opportunity": opportunity
    })

    return scored_gap


def score_all_gaps(
    revenue_gaps: List[Dict[str, Any]],
    revenue_recovery_opportunities: List[Dict[str, Any]],
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    churn_risk_customers: List[Dict[str, Any]],
    scoring_weights: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Score all revenue gaps.

    Returns:
        List of scored gaps
    """
    # Create lookup dictionaries
    opportunity_lookup = {o["customer_id"]: o for o in revenue_recovery_opportunities}
    baseline_lookup = customer_revenue_baseline
    churn_risk_lookup = {c["customer_id"]: c for c in churn_risk_customers}

    scored_gaps = []

    for gap in revenue_gaps:
        customer_id = gap["customer_id"]
        opportunity = opportunity_lookup.get(customer_id, {})
        baseline = baseline_lookup.get(customer_id, {})
        churn_risk = churn_risk_lookup.get(customer_id)

        scored_gap = score_gap(gap, opportunity, baseline, churn_risk, scoring_weights)
        scored_gaps.append(scored_gap)

    return scored_gaps

